# Resume Relevance Checker - Evaluation

This notebook evaluates the resume scoring system on a test dataset.


In [ ]:
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime

# Import our modules
from scoring.parser import extract_text_from_file
from scoring.skill_extractor import extract_skills_from_jd
from scoring.scoring import calculate_relevance_score
from scoring.feedback import generate_feedback


## Load Test Data

Load sample job descriptions and resumes for evaluation.


In [ ]:
# Sample job descriptions
sample_jds = {
    "data_scientist": """
    We are looking for a Senior Data Scientist with the following requirements:
    
    Required Skills:
    - Python programming (3+ years)
    - Machine Learning and Deep Learning
    - SQL and database management
    - Statistical analysis
    - Must have experience with pandas and numpy
    
    Preferred Skills:
    - TensorFlow or PyTorch
    - AWS or cloud platforms
    - Docker and Kubernetes
    - Git version control
    - Strong communication skills
    """,
    
    "software_engineer": """
    We need a Software Engineer with the following qualifications:
    
    Required:
    - Java or Python programming
    - Web development experience
    - Database knowledge (SQL)
    - Git version control
    - Problem-solving skills
    
    Nice to have:
    - React or Angular
    - Spring Boot or Django
    - Docker experience
    - AWS knowledge
    - Agile methodology
    """
}

# Sample resumes (mock data for testing)
sample_resumes = {
    "resume_1": """
    John Doe
    Software Developer
    
    Experience:
    - 3 years Python development
    - Machine learning projects
    - SQL and database design
    - Git and version control
    - Strong problem-solving skills
    
    Skills: Python, SQL, Machine Learning, Git, Pandas, NumPy
    """,
    
    "resume_2": """
    Jane Smith
    Data Analyst
    
    Experience:
    - 2 years data analysis
    - Excel and basic Python
    - SQL queries
    - Statistical analysis
    
    Skills: Excel, Python (basic), SQL, Statistics
    """,
    
    "resume_3": """
    Mike Johnson
    Full Stack Developer
    
    Experience:
    - 4 years web development
    - Java and Spring Boot
    - React and JavaScript
    - Database design
    - Docker and AWS
    
    Skills: Java, JavaScript, React, Spring Boot, SQL, Docker, AWS
    """
}

print(f"Loaded {len(sample_jds)} job descriptions and {len(sample_resumes)} resumes")


## Extract Skills from Job Descriptions


In [ ]:
# Extract skills from each JD
jd_skills = {}

for jd_name, jd_text in sample_jds.items():
    print(f"\nExtracting skills from {jd_name}...")
    skills = extract_skills_from_jd(jd_text)
    jd_skills[jd_name] = skills
    
    print(f"Must Have: {skills['must_have']}")
    print(f"Good to Have: {skills['good_to_have']}")


## Run Scoring Evaluation


In [ ]:
# Run scoring for all combinations
results = []

for jd_name, jd_text in sample_jds.items():
    for resume_name, resume_text in sample_resumes.items():
        print(f"\nScoring {resume_name} against {jd_name}...")
        
        try:
            score_result = calculate_relevance_score(
                jd_text=jd_text,
                resume_text=resume_text,
                jd_skills=jd_skills[jd_name],
                hard_weight=0.6,
                soft_weight=0.4
            )
            
            # Generate feedback
            feedback = generate_feedback(jd_text, resume_text, score_result.get('missing_skills', []))
            
            result = {
                'jd_name': jd_name,
                'resume_name': resume_name,
                'hard_pct': score_result.get('hard_pct', 0),
                'soft_pct': score_result.get('soft_pct', 0),
                'final_score': score_result.get('final_score', 0),
                'verdict': score_result.get('verdict', 'Low'),
                'missing_skills': score_result.get('missing_skills', []),
                'feedback': feedback
            }
            
            results.append(result)
            
            print(f"  Score: {score_result.get('final_score', 0):.1f}% ({score_result.get('verdict', 'Low')})")
            print(f"  Missing: {', '.join(score_result.get('missing_skills', [])[:3])}")
            
        except Exception as e:
            print(f"  Error: {e}")
            results.append({
                'jd_name': jd_name,
                'resume_name': resume_name,
                'hard_pct': 0,
                'soft_pct': 0,
                'final_score': 0,
                'verdict': 'Error',
                'missing_skills': [],
                'feedback': f'Error: {e}'
            })

print(f"\nCompleted scoring for {len(results)} combinations")
